In [2]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage import filters, morphology, io
import shutil

In [ ]:
file_path = '/data/nas/Cth/MRI_segmentation/'
file_list = os.listdir(file_path)
print(file_list)

In [ ]:
size_path = '/data/nas/Cth/size_total'
size_list = os.listdir(size_path)
print(size_list)

In [5]:
# Loading real path
def tar_space(png):
    file_path = '/data/nas/Cth/MRI_segmentation/'
    file_path_01 = os.path.join(file_path,png,f"{png}_nuc_t1_resample_axial2d")
    file_path_02 = os.path.join(file_path,png,"GT_rCSF_apLVc5_hippCSF")
    # print(file_path_01,file_path_02)
    return file_path_01,file_path_02

In [6]:
# Control ostu area
def create_brain_mask(image, adjustment_factor=0.2):
    threshold = filters.threshold_otsu(image)
    adjusted_threshold = threshold * adjustment_factor
    binary_mask = image > adjusted_threshold
    labeled_array, num_features = ndimage.label(binary_mask)
    sizes = ndimage.sum(binary_mask, labeled_array, range(num_features + 1))
    mask = sizes == sizes.max()
    brain_mask = mask[labeled_array]
    convex_hull = morphology.convex_hull_image(brain_mask)

    # Make rect mask
    rows = np.any(convex_hull, axis=1)
    cols = np.any(convex_hull, axis=0)
    min_row, max_row = np.where(rows)[0][[0, -1]]
    min_col, max_col = np.where(cols)[0][[0, -1]]
    rectangular_mask = np.zeros_like(image, dtype=bool)
    rectangular_mask[min_row:max_row+1, min_col:max_col+1] = True
    
    return rectangular_mask

In [7]:
# Crop mask
def crop_mask(image, mask):
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    min_row, max_row = np.where(rows)[0][[0, -1]]
    min_col, max_col = np.where(cols)[0][[0, -1]]
    cropped_image = image[min_row:max_row+1, min_col:max_col+1]
    
    return cropped_image

In [8]:
# Make new .nii file
def make_nii(or_nii_data,new_nii_data,save_path,file_name):
    output_nii_file = os.path.join(save_path, f"{file_name}.nii")
    extracted_nii = nib.Nifti1Image(new_nii_data, or_nii_data.affine)
    nib.save(extracted_nii, output_nii_file)

In [9]:
# Make new .png file
def make_png(new_nii_data, save_path, file_name):
    fig, ax = plt.subplots(figsize=(new_nii_data.shape[1] / 100, new_nii_data.shape[0] / 100), dpi=100)
    ax.imshow(new_nii_data, cmap='gray')
    ax.axis('off')
    output_png_file = os.path.join(save_path, f"{file_name}.png")
    plt.savefig(output_png_file, bbox_inches='tight', pad_inches=0)
    plt.close(fig)


In [10]:
# Padding
def pading(crop):
    sq_size = 0
    if crop.shape[0] > crop.shape[1]:
        sq_size = crop.shape[0]
        pad_r = int((sq_size - crop.shape[0])/2)
        pad_c = int((sq_size - crop.shape[1])/2)
        if ((sq_size-crop.shape[1]) % 2) == 1:
            pad = np.pad(crop,((pad_r,pad_r),(pad_c,pad_c+1)),'constant',constant_values=0)
        else:
            pad = np.pad(crop,((pad_r,pad_r),(pad_c,pad_c)),'constant',constant_values=0)

    else:
        sq_size = crop.shape[1]
        pad_r = int((sq_size - crop.shape[0])/2)
        pad_c = int((sq_size - crop.shape[1])/2)

        if ((sq_size-crop.shape[1]) % 2) == 1:
            pad = np.pad(crop,((pad_r,pad_r+1),(pad_c,pad_c)),'constant',constant_values=0)
        else:
            pad = np.pad(crop,((pad_r,pad_r),(pad_c,pad_c)),'constant',constant_values=0)

    return pad

In [11]:
# Move GT
def move_GT(fp2,fp2_files,fp4_work_n,fp4_work_p):
    for nii in fp2_files:
        if nii.endswith('.nii'):
            nii_path_2 = os.path.join(fp2,nii)
            shutil.copy(nii_path_2,os.path.join(fp4_work_n,nii))

In [ ]:
# Main code

# Decide the work place
for png_size in size_list:
    print(png_size)
    png_path = os.path.join(size_path, png_size)
    png_list = os.listdir(png_path)

    for png_name in png_list:
        print(png_name)
        png = png_name.split('.')[0] # working file name

        work_space = os.path.join('/data/nas/Cth/image_space/',png_size,png)
        # os.makedirs(work_space,exist_ok=True)
        
        # PNG path
        fp1_work_p = os.path.join(work_space,'PNG',f'{png}_SQ_PNG') # saving loc SQ
        fp2_work_p = os.path.join(work_space,'PNG',f'{png}_PAD_PNG') # saving loc PAD
        fp3_work_p = os.path.join(work_space,'PNG',f'{png}_OR_PNG') # Real original image
        fp4_work_p = os.path.join(work_space,'PNG',f'{png}_GTOR_PNG') # GT original image
        fp5_work_p = os.path.join(work_space,'PNG',f'{png}_GTSQ_PNG') # saving loc GTSQ
        fp6_work_p = os.path.join(work_space,'PNG',f'{png}_GTPAD_PNG') # saving loc GT PAD
        os.makedirs(fp1_work_p,exist_ok=True)
        os.makedirs(fp2_work_p,exist_ok=True)
        os.makedirs(fp3_work_p,exist_ok=True)
        os.makedirs(fp4_work_p,exist_ok=True)
        os.makedirs(fp5_work_p,exist_ok=True)
        os.makedirs(fp6_work_p,exist_ok=True)

        # NII path
        fp1_work_n = os.path.join(work_space,'NII',f'{png}_SQ_NII') # saving loc SQ
        fp2_work_n = os.path.join(work_space,'NII',f'{png}_PAD_NII') # saving loc PAD
        fp3_work_n = os.path.join(work_space,'NII',f'{png}_OR_NII') # Real original nii
        fp4_work_n = os.path.join(work_space,'NII',f'{png}_GTOR_NII') # GT original nii
        fp5_work_n = os.path.join(work_space,'NII',f'{png}_GTSQ_NII') # saving loc GTSQ
        fp6_work_n = os.path.join(work_space,'NII',f'{png}_GTPAD_NII') # saving loc GT PAD
        os.makedirs(fp1_work_n,exist_ok=True)
        os.makedirs(fp2_work_n,exist_ok=True)
        os.makedirs(fp3_work_n,exist_ok=True)
        os.makedirs(fp4_work_n,exist_ok=True)
        os.makedirs(fp5_work_n,exist_ok=True)
        os.makedirs(fp6_work_n,exist_ok=True)

        fp1,fp2 = tar_space(png)
        fp1_files = os.listdir(fp1)
        fp2_files = os.listdir(fp2)
    
        move_GT(fp2,fp2_files,fp4_work_n,fp4_work_p)

        fp4_files = os.listdir(fp4_work_n)
        
        # Find biggest mask
        mask = None
        mask_area = 0
        for nii in fp1_files:
            # Load Original.nii
            nii_path_1 = os.path.join(fp1,nii)
            nii_data_1 = nib.load(nii_path_1)
            nii_data_1 = nii_data_1.get_fdata()
            nii_data_1 = np.rot90(nii_data_1)
            nii_name = nii.split('.')[0]
            
            # Making & Choosing mask
            mask1 = create_brain_mask(nii_data_1)
            area = np.sum(mask1)
            if area > mask_area:
                mask = mask1
                mask_area = area

        for GT in fp4_files:
            nii_path_2 = os.path.join(fp4_work_n,GT)
            nii_data_2 = nib.load(nii_path_2)
            nii_data_2_nii = nii_data_2.get_fdata()
            nii_data_2_np = np.rot90(nii_data_2_nii)
            nii_name_2 = GT.split('.')[0]

            # Original file
            make_png(nii_data_2_np,fp4_work_p,nii_name_2)
            print(nii_data_2_np.shape)

            # Crop mask area
            crop = crop_mask(nii_data_2_np,mask)
            make_png(crop,fp5_work_p,nii_name_2)
            make_nii(nii_data_2,crop,fp5_work_n,nii_name_2)
            print(crop.shape)

            # Padding area
            pad = pading(crop)
            make_png(pad,fp6_work_p,nii_name_2)
            make_nii(nii_data_2,pad,fp6_work_n,nii_name_2)
            print(pad.shape)
        

        for nii in fp1_files:
            # Load Original.nii
            nii_path_1 = os.path.join(fp1,nii)
            nii_data_1 = nib.load(nii_path_1)
            nii_data_1_nii = nii_data_1.get_fdata()
            nii_data_1_np = np.rot90(nii_data_1_nii)
            nii_name = nii.split('.')[0]

            # Original file
            make_png(nii_data_1_np,fp3_work_p,nii_name)
            make_nii(nii_data_1,nii_data_1_np,fp3_work_n,nii_name)
            print(nii_data_1_np.shape)

            # Crop mask area
            crop = crop_mask(nii_data_1_np,mask)
            make_png(crop,fp1_work_p,nii_name)
            make_nii(nii_data_1,crop,fp1_work_n,nii_name)
            print(crop.shape)

            # Padding area
            pad = pading(crop)
            make_png(pad,fp2_work_p,nii_name)
            make_nii(nii_data_1,pad,fp2_work_n,nii_name)
            print(pad.shape)